In [1]:
#kafka 라이브러리 설치
!pip3 install kafka-python

     |████████████████████████████████| 246 kB 1.4 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [52]:
# 토픽이 읽을 파일 생성
with open("new-topic.txt","w")as f:
    for i in range(1,1001):
        f.write(str(i)+'\n')

In [6]:
# 프로듀서 생성(토픽으로 데이터 전송)
from kafka import KafkaProducer
import json
import time
from csv import reader

In [7]:
help(KafkaProducer)

Help on class KafkaProducer in module kafka.producer.kafka:

class KafkaProducer(builtins.object)
 |  KafkaProducer(**configs)
 |  
 |  A Kafka client that publishes records to the Kafka cluster.
 |  
 |  The producer is thread safe and sharing a single producer instance across
 |  threads will generally be faster than having multiple instances.
 |  
 |  The producer consists of a pool of buffer space that holds records that
 |  haven't yet been transmitted to the server as well as a background I/O
 |  thread that is responsible for turning these records into requests and
 |  transmitting them to the cluster.
 |  
 |  :meth:`~kafka.KafkaProducer.send` is asynchronous. When called it adds the
 |  record to a buffer of pending record sends and immediately returns. This
 |  allows the producer to batch together individual records for efficiency.
 |  
 |  The 'acks' config controls the criteria under which requests are considered
 |  complete. The "all" setting will result in blocking on t

In [50]:
class MessageProducer:
    broker,topic,producer = "","",None
    def __init__(self, broker, topic):
        self.broker = broker
        self.topic = topic
        params = {
            "bootstrap_servers":self.broker,
            "value_serializer": lambda x : json.dumps(x).encode("utf-8"),
            "acks":0, #파티션 리더에게 메시지 전달하고 전달이 되었다는 연락을 받지않고 다음 데이터를 전달하러감(높은 전송 속도는 보장하지만 위험도 높 -> 중요하지 않은 경우 사용)
            "api_version":(2,5,0), #튜플이니 콤마로 표시
            "retries":3 #재전송 횟수
        }
        self.producer = KafkaProducer(**params)
    def Send_message(self, msg):
        try:
            future = self.producer.send(self.topic, msg)
            self.producer.flush() #메세지; 버퍼를 비워줌
            future.get(timeout=60) # 60초 대기
            return{"status_code":200, "error":None}
        except Exception as e:
            print(f"error: {e}")
            return e 

In [56]:
broker = 'localhost:9092'
topic = 'TOPIC1'
message_producer = MessageProducer(broker, topic)
with open("new-topic.txt", "r", encoding="utf-8") as f:
    for data in f:
        res = message_producer.Send_message(data)